In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import numpy

In [4]:
index = faiss.read_index('doc_index.faiss')
with open('doc_split_text.pkl', 'rb') as f:
    split_text = pickle.load(f)

In [3]:
emb_model_path = r"C:\Users\aksha\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\c9745ed1d9f207416be6d2e6f8de32d1f16199bf"

In [4]:
emb_model = SentenceTransformer(emb_model_path)

In [5]:
query = "What are the main findings from the document"
query_emb = emb_model.encode([query]).astype('float32')

In [6]:
k = 3 
D, I = index.search(query_emb, k)

In [7]:
retrieved_chunks = [split_text[i] for i in I[0]]
context = "\n\n".join(retrieved_chunks)

In [8]:
print(context[:500])

These publications exemplify TheraCUES' dedication to advancing research in various biomedical fields. For a comprehensive list of their publications and access to full articles, please visit their Publications page.
THERACUES, ABOUT: OVERVIEW
TheraCUES, whose name combines "thera" (therapeutic) and "CUES" (signals), is dedicated to advancing translational research in precision medicine. Their mission is to expedite impactful discoveries and therapies to the market by providing top-tier solution


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_path = r"C:\Users\aksha\.cache\huggingface\hub\models--unsloth--mistral-7b-instruct-v0.1-bnb-4bit\snapshots\ce41ab8056bfc8c399ae1429276920914c3c295e"

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto", local_files_only=True)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

prompt = f"""
You are a helpful assistant.
Use the following context to answer the question.

Context:
{context}

Question: {query}

Answer:
"""

response = pipe(prompt, max_new_tokens=300, temperature=0.3, top_p=0.9)
print("Answer:\n", response[0]['generated_text'])


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


🧠 Answer:
 
You are a helpful assistant.
Use the following context to answer the question.

Context:
These publications exemplify TheraCUES' dedication to advancing research in various biomedical fields. For a comprehensive list of their publications and access to full articles, please visit their Publications page.
THERACUES, ABOUT: OVERVIEW
TheraCUES, whose name combines "thera" (therapeutic) and "CUES" (signals), is dedicated to advancing translational research in precision medicine. Their mission is to expedite impactful discoveries and therapies to the market by providing top-tier solutions to pharmaceutical companies, academic institutions, and biotech firms.

Extraction Services: Understanding the importance of accurate nucleic acid isolation, TheraCUES offers services to ensure RNA and DNA extractions are performed with precision, maintaining integrity and preventing contamination. This service is crucial for the success of molecular biology assays.
Custom Solutions: TheraCUES 

In [ ]:
def ask_question(query, top_k=3):

    query_emb = emb_model.encode([query]).astype("float32")

    D, I = index.search(query_emb, top_k)
    retrieved = [split_text[i] for i in I[0]]
    context = "\n\n".join(retrieved)

    prompt = f"""
    You are a helpful assistant.
    Use the following context to answer the question.

    Context:
    {context}

    Question: {query}

    Answer:
    """

    response = pipe(
        prompt,
        max_new_tokens=300,
        temperature=0.3,
        top_p=0.9
    )[0]['generated_text']

    if "Answer:" in response:
        response = response.split("Answer:")[-1].strip()

    return f"Question: {query}\n\nAnswer: {response}"


In [23]:
print(ask_question("What does nCounter do"))

Question: What does nCounter do

Answer: nCounter is a technology developed by NanoString that enables the simultaneous measurement of approximately 800 gene expression targets. It offers several advantages over traditional methods like quantitative PCR (qPCR) and next-generation sequencing (NGS), including high throughput, reproducible data, and speed. TheraCUES, a company that utilizes nCounter technology, provides end-to-end assistance in gene expression profiling, from nucleic acid extraction to advanced data analysis, delivering actionable biological insights within 1-3 weeks, depending on sample size and project complexity.
